In [ ]:
from huggingface_hub import login
login(token="") 

In [ ]:
!pip install --upgrade transformers
!pip install evaluate
!pip install rouge_score
!pip install bert_score
!pip install torch --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 70.9 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.

In [ ]:
import re
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments

In [ ]:
# Load the dataset
dataset = load_dataset("omi-health/medical-dialogue-to-soap-summary")
print("Original Training Split Size:", len(dataset["train"]))
print("Original Validation Split Size:", len(dataset["validation"]))
print("Original Test Split Size:", len(dataset["test"]))

# Preprocessing function
def preprocess_dialogue(example):
    dialogue = example["dialogue"]
    soap = example["soap"]

    # Clean text
    dialogue = re.sub(r'[^A-Za-z0-9\s.,:?-]', '', dialogue).lower()
    soap = re.sub(r'[^A-Za-z0-9\s.,:?-]', '', soap).lower()

    # Add role tags
    dialogue = dialogue.replace("Doctor:", "[Doctor]:")
    dialogue = dialogue.replace("Patient:", "[Patient]:")

    return {"dialogue": dialogue, "soap": soap}

# Apply preprocessing
processed_dataset = dataset.map(preprocess_dialogue)

Original Training Split Size: 9250
Original Validation Split Size: 500
Original Test Split Size: 250


In [ ]:
# Load BART tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")

# Tokenization function for dialogue-to-SOAP task
def tokenize_function(example):
    # Tokenize dialogue (input)
    model_inputs = tokenizer(
        example["dialogue"],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )

    # Tokenize SOAP (target)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["soap"],
            truncation=True,
            padding="max_length",
            max_length=256,  # Shorter for SOAP as it's more concise
            return_tensors="pt"
        )

    model_inputs["labels"] = labels["input_ids"].squeeze()
    return model_inputs

# Apply tokenization
tokenized_dataset = processed_dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns and set format
tokenized_dataset = tokenized_dataset.remove_columns(["dialogue", "soap"])
tokenized_dataset.set_format("torch")

# Split into train and eval datasets
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["validation"]

# Debug: Verify dataset
print("Sample from train_dataset:", train_dataset[0].keys())
print("Sample 'input_ids' shape:", train_dataset[0]["input_ids"].shape)
print("Sample 'labels' shape:", train_dataset[0]["labels"].shape)
print("Sample from eval_dataset:", eval_dataset[0].keys())
print("Sample 'labels' shape (eval):", eval_dataset[0]["labels"].shape)

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Sample from train_dataset: dict_keys(['prompt', 'messages', 'messages_nosystem', 'input_ids', 'attention_mask', 'labels'])
Sample 'input_ids' shape: torch.Size([512])
Sample 'labels' shape: torch.Size([256])
Sample from eval_dataset: dict_keys(['prompt', 'messages', 'messages_nosystem', 'input_ids', 'attention_mask', 'labels'])
Sample 'labels' shape (eval): torch.Size([256])


In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForSeq2SeqLM, AutoTokenizer

# Load BART model
# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")
model= AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large", use_safetensors=True)

# Freeze BART's encoder for feature extraction
for param in model.model.encoder.parameters():
    param.requires_grad = False

# Optionally, freeze all but the last decoder layer
for layer in model.model.decoder.layers[:-1]:
    for param in layer.parameters():
        param.requires_grad = False

# Define training arguments
training_args = TrainingArguments(
    output_dir="./bart-soap-finetuned",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True,
    per_device_train_batch_size=8,  # Increased from 4 to 8
    per_device_eval_batch_size=8,   # Increased from 4 to 8
    num_train_epochs=3,
    weight_decay=0.01,
    learning_rate=1e-4,  # Increased learning rate to match larger batch size
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none",
    gradient_accumulation_steps=2,  # Added to effectively increase batch size further
    warmup_steps=500,  # Added warmup steps for better training stability
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

/tmp/ipykernel_31/689094349.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train the model
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.053600,1.498606
2,1.535000,1.264419


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight

TrainOutput(global_step=867, training_loss=1.796726873589222, metrics={'train_runtime': 2817.5165, 'train_samples_per_second': 9.849, 'train_steps_per_second': 0.308, 'total_flos': 2.9997061902434304e+16, 'train_loss': 1.796726873589222, 'epoch': 2.991364421416235})

In [ ]:

# Save the fine-tuned model
model.save_pretrained("./bart-soap-finetuned-final")
tokenizer.save_pretrained("./bart-soap-finetuned-final")


('./bart-soap-finetuned-final/tokenizer_config.json',
 './bart-soap-finetuned-final/special_tokens_map.json',
 './bart-soap-finetuned-final/vocab.json',
 './bart-soap-finetuned-final/merges.txt',
 './bart-soap-finetuned-final/added_tokens.json',
 './bart-soap-finetuned-final/tokenizer.json')

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load the fine-tuned model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("./bart-soap-finetuned-final")
tokenizer = AutoTokenizer.from_pretrained("./bart-soap-finetuned-final")

# Inference function
def generate_soap(dialogue, model, tokenizer, device="cuda" if torch.cuda.is_available() else "cpu"):
    model.to(device)
    inputs = tokenizer(
        dialogue,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=512
    ).to(device)
    
    # Generate SOAP report
    outputs = model.generate(
        inputs["input_ids"],
        max_length=256,
        num_beams=4,
        early_stopping=True
    )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test inference on a sample (you can replace this with any input dialogue)
sample_dialogue = processed_dataset["test"][2]["dialogue"]
sample_soap = processed_dataset["test"][2]["soap"]
print("\nSample Dialogue:", sample_dialogue)
print("\n\nGenerated SOAP:", generate_soap(sample_dialogue, model, tokenizer))
print("\n\nReference SOAP:", sample_soap)


Sample Dialogue: doctor: hello, weve received your results from the ultrasound we performed in april 2017. it seems that there is a single thyroid nodule present in your left lobe, measuring 1 cm in its largest diameter. we also performed a complete biochemical screening, including tests for tsh, autoantibodies, and calcitonin.
patient: hmm, what did the screening results show, doctor?
doctor: your calcitonin level was found to be slightly elevated at 40 ngml, which is above the normal range of 1-4.8 ngml. to further investigate, we performed a stimulation test with intravenous calcium.
patient: and what did the stimulation test show?
doctor: after the stimulation, your calcitonin levels peaked at 1420 ngml, which indicates that surgical treatment is necessary. as a result, you underwent a total thyroidectomy and central neck dissection on the side of the tumor.
patient: yes, i remember that. how was my recovery after the surgery?
doctor: your postoperative course was uneventful, with

In [ ]:
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load the fine-tuned model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("./bart-soap-finetuned-final")
tokenizer = AutoTokenizer.from_pretrained("./bart-soap-finetuned-final")

# Inference function
def generate_soap(dialogue, model, tokenizer, device="cuda" if torch.cuda.is_available() else "cpu"):
    model.to(device)
    model.eval()
    inputs = tokenizer(
        dialogue,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=512
    ).to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_length=256,
            num_beams=4,
            early_stopping=True
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Generate and collect results for the first 100 samples
device = "cuda" if torch.cuda.is_available() else "cpu"

results = []

for idx in range(250):
    sample_dialogue = processed_dataset["test"][idx]["dialogue"]
    reference_soap = processed_dataset["test"][idx]["soap"]
    
    generated_soap = generate_soap(sample_dialogue, model, tokenizer, device=device)
    
    results.append({
        "Dialogue": sample_dialogue,
        "Reference SOAP": reference_soap,
        "Generated SOAP": generated_soap
    })

# Save the results to a CSV file
df = pd.DataFrame(results)
df.to_csv("generated_soap_results.csv", index=False)

print("Results saved to 'generated_soap_result.csv' ✅")


Results saved to 'generated_soap_results.csv' ✅


In [ ]:
import evaluate
import pandas as pd

# Load your saved CSV
df = pd.read_csv("/kaggle/working/generated_soap_results.csv")

# Extract generated and reference lists
generated_soap_list = df["Generated SOAP"].tolist()
reference_soap_list = df["Reference SOAP"].tolist()

# Load evaluation metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")

# Compute ROUGE
rouge_result = rouge.compute(predictions=generated_soap_list, references=reference_soap_list)
print("\nROUGE Results:")
print(rouge_result)

# Compute BLEU
# BLEU expects references as a list of lists
bleu_result = bleu.compute(predictions=generated_soap_list, references=[[ref] for ref in reference_soap_list])
print("\nBLEU Results:")
print(bleu_result)

# Compute BERTScore/
bertscore_result = bertscore.compute(predictions=generated_soap_list, references=reference_soap_list, lang="en")
print("\nBERTScore Results:")
print({
    "precision": sum(bertscore_result["precision"]) / len(bertscore_result["precision"]),
    "recall": sum(bertscore_result["recall"]) / len(bertscore_result["recall"]),
    "f1": sum(bertscore_result["f1"]) / len(bertscore_result["f1"])
})



ROUGE Results:
{'rouge1': 0.5309840447763023, 'rouge2': 0.29942434276083774, 'rougeL': 0.3452967377472327, 'rougeLsum': 0.45044177469003555}

BLEU Results:
{'bleu': 0.20601458838890105, 'precisions': [0.7140564039683425, 0.42934648581997537, 0.28411678502987264, 0.19743793220723274], 'brevity_penalty': 0.5688935314973022, 'length_ratio': 0.6393608529562689, 'translation_length': 44855, 'reference_length': 70156}


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



BERTScore Results:
{'precision': 0.9084752750396728, 'recall': 0.8776876709461212, 'f1': 0.8927354657649994}
